# Исследование надёжности заёмщиков

ПРОЕКТНАЯ РАБОТА - СПРИНТ 2

### Содержание
1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [Обработка пропусков](#null)
    * [Замена типа данных](#data_type)
    * [Обработка дубликатов](#duble)
    * [Лемматизация](#lemmatiz)
    * [Категоризация](#categor)
3. [Аналитика](#analitika)
4. [Общий вывод](#general)



Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.Состоящая из:  
1. Количества детей
2. Стажа
3. Возраста
4. Образования
5. Семейного положения
6. Пола
7. Формы занятости
8. Наличия факта просрочки
9. Дохода
10. Цели кредита

**Данные предоставлены в формате csv, выборка содержит порядка 21,5 тысячи записей.**

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.
<a name="title"></a>



<a name="start"></a>
### Открытие данных. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
df = pd.read_csv('datasets/data.csv')
df.info()
df.describe(include = 'all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
count,21525.000000,19351.000000,21525.000000,21525,21525.000000,21525,21525.000000,21525,21525,21525.000000,1.935100e+04,21525
unique,NaN,NaN,NaN,15,NaN,5,NaN,3,8,NaN,NaN,38
top,NaN,NaN,NaN,среднее,NaN,женат / замужем,NaN,F,сотрудник,NaN,NaN,свадьба
freq,NaN,NaN,NaN,13750,NaN,12380,NaN,14236,11119,NaN,NaN,797
mean,0.538908,63046.497661,43.293380,NaN,0.817236,NaN,0.972544,NaN,NaN,0.080883,1.674223e+05,NaN
std,1.381587,140827.311974,12.574584,NaN,0.548138,NaN,1.420324,NaN,NaN,0.272661,1.029716e+05,NaN
min,-1.000000,-18388.949901,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,2.066726e+04,NaN
25%,0.000000,-2747.423625,33.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.030532e+05,NaN
50%,0.000000,-1203.369529,42.000000,NaN,1.000000,NaN,0.000000,NaN,NaN,0.000000,1.450179e+05,NaN
75%,1.000000,-291.095954,53.000000,NaN,1.000000,NaN,1.000000,NaN,NaN,0.000000,2.034351e+05,NaN


In [3]:
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


### Вывод

0. Таблица 21.5 тыс.записей, 12 столбцов, к именам столбцов замечаний нет, имеются пропуски по 2 столбцам.
1. По типу данных - вопрос вызывает только 'debt' - думаю можно заменить его на булин. Так же думаю не правильно использовать float для столбца 'days_employed', больше подходит int, но в этом столбце вообще твориться какой-то бред: минусы пол-беды, а вот 932 года стажа это круто... Надеюсь, что он мне для решения данного кейса не понадобиться)))
2. По пропускам пока понимания нет: видно что пропуски в двух стобцах - стаже и доходе, и их одинаковое количество... Если они полностью совподают по строкам, то это логично и можно будет "занулить" доход для тех кто не работает. Глядишь хоть столбец 'days_employed' на что сгодится. Но это посмотрим ниже...
3. Есть разбежка в вариантах написаниях в 'education', ну и раз есть здесь, то надо проверить и все остальные текстовые записи.

[к содержанию...](#title)

<a name="preprocessing"></a>
### Предобработка данных

<a name="null"></a>
### Обработка пропусков

In [4]:
print('Количество пропущенных значение в каждом  из двух столбцов', 21525 - 19351)
print('Они все совподают построчно:')
df[df['days_employed'].isnull()] == df[df['total_income'].isnull()]


Количество пропущенных значение в каждом  из двух столбцов 2174
Они все совподают построчно:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,True,False,True,True,True,True,True,True,True,True,False,True
26,True,False,True,True,True,True,True,True,True,True,False,True
29,True,False,True,True,True,True,True,True,True,True,False,True
41,True,False,True,True,True,True,True,True,True,True,False,True
55,True,False,True,True,True,True,True,True,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,True,False,True,True,True,True,True,True,True,True,False,True
21495,True,False,True,True,True,True,True,True,True,True,False,True
21497,True,False,True,True,True,True,True,True,True,True,False,True
21502,True,False,True,True,True,True,True,True,True,True,False,True


In [5]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


[к содержанию...](#title)

### Вывод

Учитывая, что значения NaN в столбцах 'day_employed' и 'total_income' полность совподают построчно, целесообразно не удалять их, а заменить на 0. Предположим, что это клиенты "без справок о доходах"

<a name="data_type"></a>
### Замена типа данных

In [6]:
df['debt'] = df['debt'].astype('bool')
df['days_employed'] = df['days_employed'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  bool   
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 1.8+ MB


### Вывод

Заменил тип переменной столбца 'debt' на bool - "имел или не имел"
По остальным столбцам меня все устраивает...
В принципе можно было и этот не трогать, но больше не знаю что менять.

[к содержанию...](#title)

<a name="duble"></a>
### Обработка дубликатов

In [7]:
df['education'] = df['education'].str.lower()
print(df['education'].unique())
print('-----------------------------')

df['family_status'] = df['family_status'].str.lower()
print(df['family_status'].unique())
print('-----------------------------')

df['income_type'] = df['income_type'].str.lower()
print(df['income_type'].unique())
print('-----------------------------')

df['purpose'] = df['purpose'].str.lower()
print(df['purpose'].unique())
print('-----------------------------')

print(df['gender'].unique())
print('-----------------------------')


print('полных дубликатов - ', df.duplicated().sum())
#df = df.drop_duplicates().reset_index(drop=True)
print('а теперь дубликатов - ', df.duplicated().sum())
df.info()

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
-----------------------------
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
-----------------------------
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
-----------------------------
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' '

### Вывод

В столбцах:'education', 'family_status', 'income_type' просто перевел все к нижнему регистру, все значения адекватны, дублей нет. Думаю делать это в принципе было не обязательно, так как везде есть информация о id в "цифре", но для порядка и тренировки. С purpose так не вышло, его будем лематизировать... 
df.duplicated().sum() на всякий случай сделал там 0.
ПС: На всякий случай проверил 'gender' и нашел там "ХНу" - надо проверить что это и сколько его.

[к содержанию...](#title)

<a name="lemmatiz"></a>
### Лемматизация

In [8]:
from pymystem3 import Mystem
m = Mystem()

def purpose_type(query):
    query_lem =  m.lemmatize(query)   
    if 'жилье' in query_lem:
        return 'недвижимость'
    if 'недвижимость' in query_lem:
        return 'недвижимость'
    if 'образование' in query_lem:
        return 'образование'
    if 'автомобиль' in query_lem:
        return 'авто'
    if 'свадьба' in query_lem:
        return 'свадьба'
    return 'неопределен'
    
#purpose_type(df.loc[1657, 'purpose'])

df['gl_purpose'] = df['purpose'].apply(purpose_type)

print(df['gl_purpose'].unique())
print('-----------------------------')
print('')
df.info()


['недвижимость' 'авто' 'образование' 'свадьба']
-----------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  bool   
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  gl_purpose        21525 non-null  object 
dtypes: bool(1), float64(1), int64(5), object(6)
memory usage: 2.0+ MB


Вывод¶

Первоначально для столбца 'purpose', напрашиваются следующие цели: недвижимость, авто, образование, свадьба. В таком ключе и пойдем, создав новый столбец 'gl_purpose'. Проверил чтобы не получилось NaN и 'неопределен'. 
ПС: Заодно и категоризацию замутил сразу)))

[к содержанию...](#title)

<a name="categor"></a>
### Категоризация данных

In [9]:
def total_type(query):
    if query == 0:
        return 'не указан'
    if query <= 15000:
        return 'минимальный'
    if 15000 < query <= 30000:
        return 'низкий'
    if 30000 < query <= 100000:
        return 'средний'
    if query > 100000:
        return 'высокий'
    return 'неопределен'
    
df['total_type'] = df['total_income'].apply(total_type)

print(df['total_type'].unique())
print('-----------------------------')
print('')
df.info()


['высокий' 'средний' 'не указан' 'низкий']
-----------------------------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  bool   
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  gl_purpose        21525 non-null  object 
 13  total_type        21525 non-null  object 
dtypes: bool(1), float64(1), int64(5), object(7)
memory usage: 2.

### Вывод

У нас уже есть категории в "Целях кредита", так же пригодится расскидать категории по уровню дохода. Сделаем так: не указан = 0, до 15 - минимальный, от 15 до 30 - будет низкий, до 100 средний, от 100 высокий.

[к содержанию...](#title)

<a name="analitika"></a>
### Аналитика

#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
age_pivot_sum = df.groupby('children')['debt'].sum()
age_pivot_count = df.groupby('children')['debt'].count()
age_pivot = age_pivot_sum / age_pivot_count
print(age_pivot_count)
print(age_pivot.sort_values())

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: debt, dtype: int64
children
 5     0.000000
-1     0.021277
 0     0.075129
 3     0.081818
 1     0.092154
 2     0.094404
 4     0.097561
 20    0.105263
Name: debt, dtype: float64


### Вывод

Если честно я ожидал обратного, но:
.............
Наличие детей приводит к увеличению вероятности возникновения задолженности.
.............
Из анализа исключил данные с количеством детей -1 и 20 (буду считать это ошибкой, хотя конечно сам результат и там, и там очень интересно выбивается, но я к сожалению не пойму пока за что его зацепить). Так же выкинул из анализа 4-х и 5-х детей из-за низкой представленности. 
.............
А в целом получается, что: Клиенты не имеющие детей попадают в просрочку в 7.5% случаев, в то время клиенты имеющие 1 ил 2 детей уже в 9.3%. Далее по мере роста количества детей вероятность просрочки начинает снижаться и для 3 детей составляет 8.1%

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
fam_pivot_sum = df.groupby('family_status')['debt'].sum()
fam_pivot_count = df.groupby('family_status')['debt'].count()
fam_pivot = fam_pivot_sum / fam_pivot_count
print(fam_pivot.sort_values())


family_status
вдовец / вдова           0.065625
в разводе                0.071130
женат / замужем          0.075202
гражданский брак         0.092890
не женат / не замужем    0.097405
Name: debt, dtype: float64


### Вывод

Наиболее надежными с точки зрения семейного положения являются клиенты состоящие или состоявшие в официальном браке, менее надежными - клиенты не имеющие постоянных отношений.

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
total_pivot_sum = df.groupby('total_type')['debt'].sum()
total_pivot_count = df.groupby('total_type')['debt'].count()
total_pivot = total_pivot_sum / total_pivot_count
print(total_pivot.sort_values())

total_type
не указан    0.078197
средний      0.079261
высокий      0.081744
низкий       0.090909
Name: debt, dtype: float64


### Вывод

Проблемы с возвратом кредита чаще возникаю у клиентов имеющих доход ниже 30 тыс.рублей (9.1%). По остальным группам существенного разброса не отмечается и показатель колеблется на уровне 8%.

#### Как разные цели кредита влияют на его возврат в срок?

In [13]:
purpose_pivot_sum = df.groupby('gl_purpose')['debt'].sum()
purpose_pivot_count = df.groupby('gl_purpose')['debt'].count()
purpose_pivot = purpose_pivot_sum / purpose_pivot_count
print(purpose_pivot.sort_values())

gl_purpose
недвижимость    0.072140
свадьба         0.079216
образование     0.091994
авто            0.093395
Name: debt, dtype: float64


### Вывод

Наименее рискованными целями кредита являются операции с недвижимостью - 7.2%, скорее всего это связанно с более взвешенным решением и большей ответственностью клиента в это сфере.
Наиболее рискованными являются сделки с автомобильным транспортом - 9.3% и образовательные кредиты - 9.2%.
Относительно высокий показатель "свадебных" кредитов обусловлен скорее всего тем, что их большая часть гасится сразу после торжества.

[к содержанию...](#title)

<a name="general"></a>
### 4. Общий вывод

В целом наличие зависимостей выявлено по всем поставленным вопросам.

Краткое резюме и проценты случаев возникновения задолженности по обозначенным темам:

1. Наличие детей приводит к увеличению вероятности возникновения задолженности с 7.5% (нет детей) до 9.5% (2 ребенка).
2. Наиболее надежными с точки зрения семейного положения являются клиенты состоящие или состоявшие в официальном браке (всего около 7%). Менее надежными клиенты, не имеющие постоянных отношений (до 9.7%).
3. Проблемы с возвратом кредита чаще возникаю у клиентов имеющих доход ниже 30 тыс.рублей (9.1%).
4. Наименее рискованными целями кредита являются операции с недвижимостью - 7.2%.
Учитывая средний процент по всей выборке 8.1%, можно сделать вывод, что выбранные показатели оказывают серьезное влияние на вероятность возникновения задолженности.

[к содержанию...](#title)

# Исследование надёжности заёмщиков — анализ банковских данных

### Задачи проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.  
**Входные данные** — статистика о платёжеспособности клиентов.

### Стек
Python, Pandas, PyMystem3, лемматизация, предобработка данных


### Описание решения 
На основе данных кредитного отдела банка исследовал влияние семейного положения и количества детей на факт погашения кредита в срок. Была получена информация о
данных. Определены и обработаны пропуски. Заменены типы данных на cоответствующие хранящимся данным. Удалены дубликаты. Выделены леммы в значениях столбца и категоризированны данные.


### Общий вывод
В целом наличие зависимостей выявлено по всем поставленным вопросам:

Наличие детей приводит к увеличению вероятности возникновения задолженности с 7.5% (нет детей) до 9.5% (2 ребенка).  
Наиболее надежными с точки зрения семейного положения являются клиенты состоящие или состоявшие в официальном браке (всего около 7%). Менее надежными клиенты, не имеющие постоянных отношений (до 9.7%).  
Проблемы с возвратом кредита чаще возникаю у клиентов имеющих доход ниже 30 тыс.рублей (9.1%).  
Наименее рискованными целями кредита являются операции с недвижимостью - 7.2%. Учитывая средний процент по всей выборке 8.1%, можно сделать вывод, что выбранные показатели оказывают серьезное влияние на вероятность возникновения задолженности.